In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [2]:
#Carregar a tabela de buscas de produtos:
produtos_df = pd.read_excel(r'Base de Buscas\buscas.xlsx')
display(produtos_df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [3]:
#Abrir o site do Google Shopping:
navegador.get("https://shopping.google.com.br/")

In [4]:
#Buscar o produto:
navegador.find_element(By.ID, 'REsRA').send_keys(produtos_df.loc[0, 'Nome'])
navegador.find_element(By.CLASS_NAME, 'uDgUL').click()

In [5]:
preco_minimo = float(produtos_df.loc[0, 'Preço mínimo'])
preco_maximo = float(produtos_df.loc[0, 'Preço máximo'])

In [6]:
lista_ofertas_google = []

while True:
    # Aguardar a presença do elemento 'sh-dgr__content' antes de prosseguir:
    WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'sh-dgr__content')))
    time.sleep(1)
    
    #Pegar todos os produtos da página:
    lista_produtos = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__content')
    
    for produto in lista_produtos:
        #Obter os dados dos produtos:
        nome_produto = produto.find_element(By.TAG_NAME, 'h3').text.lower().replace(',','')
        preco_produto = produto.find_element(By.CLASS_NAME, 'a8Pemb').text.replace(' ', '').replace('R$', '').replace('.', '').replace(',', '.')
        
        #Filtrar pelos dados da busca:
        lista_termos_produto = produtos_df.loc[0, 'Nome'].lower().split(" ")
        lista_termos_banidos = produtos_df.loc[0, 'Termos banidos'].lower().split(" ")

        termo_prod_correto = all(termo in nome_produto for termo in lista_termos_produto)
        if 'iphone 11' in nome_produto:
            termo_prod_correto = False
        termo_banido_encontrado = any(termo in nome_produto for termo in lista_termos_banidos)

        if termo_prod_correto and not termo_banido_encontrado:
            if preco_minimo <= float(preco_produto) <= preco_maximo:
                link_produto = produto.find_element(By.TAG_NAME, 'a').get_attribute('href')
                link_produto = link_produto.replace('https://www.google.com.br/url?url=', '')
                lista_ofertas_google.append((nome_produto, preco_produto, link_produto))
    try:
        # Tentar clicar no botão "Mais"
        proxima_pag = navegador.find_element(By.LINK_TEXT, 'Mais')
        proxima_pag.click()
    except:
        # Caso não haja mais botão "Mais", encerrar o loop
        print('Todas as páginas percorridas')
        break

print(lista_ofertas_google)

Todas as páginas percorridas
[('vitrine: iphone 12 apple 64gb roxo tela 61 câmera traseira 12mp ios', '3286.05', 'https://www.carrefour.com.br/vitrine-iphone-12-apple-64gb-roxo-tela-61-camera-traseira-12mp-ios-mp932337971/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjxkLLZ8cL-AhUnrZUCHa0tDIcQ1SkI0AwoAA&usg=AOvVaw28NCOTzMh7If1P_ALVz8sO'), ('iphone 12 64gb - azul - estou zerado', '3059.10', 'https://www.google.com.br/shopping/product/357344828079836893?hl=pt-BR&psb=1&q=iphone+12+64gb&oq=iphone+12+64gb&gs_lcp=Cgtwcm9kdWN0cy1jYxAMUA1YJGBGaABwAHgAgAEAiAEAkgEAmAEAoAEBsAEA&sclient=products-cc&prds=eto:10360267435465518254_0,pid:4387230182954649532,rsk:PC_11630641001267601036&sa=X&ved=0ahUKEwjxkLLZ8cL-AhUnrZUCHa0tDIcQ8gII4AwoAA'), ('vitrine: iphone 12 apple 64gb azul tela 61 câmera traseira dupla 12mp', '3286.05', 'https://www.google.com.br/shopping/product/17015131505985528838?hl=pt-BR&psb=1&q=iphone+12+64gb&oq=iphone+12+64gb&gs_lcp=Cgtwcm9kdWN0cy1jYxAMUA1YJGBGaABwAHgAgAEAiAEAkgEAmAEAoAEBsAEA&sclient=pr